# Introduction

Considering that I'm a Starbucks big data analyst looking to find the next store into a [Starbucks Reserve Roastery](https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5) in California.  These roasteries are much larger than a typical Starbucks store and have several additional features, including various food and wine options, along with upscale lounge areas.  You'll investigate the demographics of various counties in the state of California, to determine potentially suitable locations.

<center>
<img src="intro_image.png" width="450"><br/><br/>
</center>

In [22]:
from geopy.geocoders import Nominatim 
import math
import pandas as pd
import geopandas as gpd
import folium 
from folium import Marker
from folium.plugins import MarkerCluster

I'll use the `embed_map()` function to visualize our maps.

In [23]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [24]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


In [25]:
starbucks[starbucks.isnull().any(axis=1)]

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


Most of the stores have known (latitude, longitude) locations.  But, all of the locations in the city of Berkeley are missing.

In [26]:
# number of rows in each column having missing values?
print(starbucks.isnull().sum())

# Viewing rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


### Using Nominatim to geocode the missing locations of the stores by their address.

In [28]:
# Create the geocoder
geolocator = Nominatim(user_agent="kaggle_learn")

# function to get latitude and longitude from an address
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Longitude': point.longitude, 'Latitude': point.latitude})
    except:
        return None

# Geocoding address to lat/long
locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']),axis=1)
starbucks.update(locations)

In [29]:
starbucks[starbucks["City"]=="Berkeley"]

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,-122.268230,37.868839
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,-122.280013,37.891477
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,-122.269869,37.881177
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,-122.259406,37.855903
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,-122.266095,37.870253


### Visualizing the (latitude, longitude) locations in Berkeley we identified in the OpenStreetMap style. 

In [45]:
# Creating a base map
m_1 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Adding a marker for each Berkeley location
# Add points to the map
for idx, row in starbucks.iterrows():
    if row['City'] == 'Berkeley':
        Marker([row['Latitude'], row['Longitude']],popup=row['Store Name']).add_to(m_1)

# Show the map
embed_map(m_1, 'q_1.html')

![Q1](q1.jpg)

#### All of the locations in Berkeley that were missing were geocoded correctly using Nominatim.

### Load a GeoDataFrame `CA_counties` containing the name, area (in square kilometers), and a unique id (in the "GEOID" column) for each county in the state of California.  The "geometry" column contains a polygon with county boundaries.

In [31]:
CA_counties = gpd.read_file("CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.crs = {'init': 'epsg:4326'}
CA_counties.head()

/Users/admin/anaconda3/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


Loading the DataFrames:
- `CA_pop` that contains an estimate of the population of each county.
- `CA_high_earners` that contains the number of households with an income of at least $150,000 per year.
- `CA_median_age` that contains the median age for each county.

In [32]:
CA_pop = pd.read_csv("CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("CA_county_median_age.csv", index_col="GEOID")

### Joining the `CA_counties` GeoDataFrame with `CA_pop`, `CA_high_earners`, and `CA_median_age`. 

In [33]:
cols_add = CA_pop.join([CA_high_earners,CA_median_age]).reset_index()
CA_stats = CA_counties.merge(cols_add,on='GEOID')
CA_stats.head()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5


### Now that we have all of the data in one place, it's much easier to calculate statistics that use a combination of columns. Let's create a "density" column with the population density.

In [34]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]

### Counties that look promising?

#### Selecting counties that meet specific criteria. In particular, I'm going to select counties where:
- there are at least 100,000 households making \$150,000 per year,
- the median age is less than 38.5, and
- the density of inhabitants is at least 285 (per square kilometer).

#### Additionally, selected counties should satisfy at least one of the following criteria:
- there are at least 500,000 households making \$150,000 per year,
- the median age is less than 35.5, or
- the density of inhabitants is at least 1400 (per square kilometer).

This is just a criteria that I think will work well for identifying the next Starbucks Reserve Roastery location.

In [35]:
CA_stats.head()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0,1.198638
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9,598.376878
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7,45.499825
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6,16.980022
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5,148.788107


In [36]:
# Selecting the rows from the DataFrame that satisfy the criterias that I have described above.
sel_counties = CA_stats[(CA_stats['high_earners'] >= 100000) & (CA_stats['median_age'] < 38.5) & (CA_stats['density'] >= 285)
                 & ((CA_stats['high_earners'] >= 500000) | (CA_stats['median_age'] < 35.5) | (CA_stats['density'] >= 1400))]

In [39]:
sel_counties

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0,821.227834
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4,285.237299
10,6075,San Francisco County,600.588247,"MULTIPOLYGON (((-122.60025 37.80249, -122.6123...",883305,114989,38.3,1470.733077


### Creating a GeoDataFrame `starbucks_gdf` with all of the starbucks locations in the selected counties that meet the criteria.

In [37]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}

/Users/admin/anaconda3/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [38]:
starbucks_gdf.head()

,Store Number,Store Name,Address,City,Longitude,Latitude,geometry
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51,POINT (-117.40000 34.51000)
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76000 34.16000)
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76000 34.15000)
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87,POINT (-117.75000 33.87000)
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79,POINT (-122.28000 37.79000)


In [40]:
# names of selected counties in California 
sel_names = sel_counties['name'].tolist()
sel_names

['Los Angeles County', 'San Diego County', 'San Francisco County']

In [41]:
# number of stores in selected counties
num_stores = len(gpd.sjoin(starbucks_gdf,sel_counties))
num_stores

1043

### Visualizing the store locations.

In [42]:
# Joining selected counties with starbucks geodataframe to get only the selected stores
sel_stores = gpd.sjoin(starbucks_gdf,sel_counties)
sel_stores.head()

,Store Number,Store Name,Address,City,Longitude,Latitude,geometry,index_right,GEOID,name,area_sqkm,population,high_earners,median_age,density
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76000 34.16000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76000 34.15000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
14,76365-97162,Target Alhambra T-184,1220 West Main Street Alhambra CA,Alhambra,-118.14,34.09,POINT (-118.14000 34.09000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
15,6794-41839,Fremont Ave & Mission Rd,"1131 S Fremont Ave, A Alhambra CA",Alhambra,-118.15,34.08,POINT (-118.15000 34.08000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
16,11220-104633,"Atlantic & Valley, Alhambra",1410 South Atlantic Blvd. Alhambra CA,Alhambra,-118.13,34.08,POINT (-118.13000 34.08000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834


In [46]:
# Creating base map
m_2 = folium.Map(location=[37,-120], zoom_start=6)

# Show selected store locations
mc = MarkerCluster()
for idx, row in sel_stores.iterrows():
    mc.add_child(Marker([row['Latitude'], row['Longitude']]))
m_2.add_child(mc)

# Show the map
embed_map(m_2, 'q_2.html')

![Q1](q2.jpg)

## I successfully identified a total of 1043 potential Starbucks store locations across the selected counties. The distribution of stores is as follows:
- Los Angeles has 667 stores. 
- San Diego has 289 stores.
- San Francisco has 87 stores. 